# S(q,$\omega$) for confluent layer of MDCK cells

Movie from [T.E. Angelini *et al*, Cell Migration Driven by Cooperative Substrate Deformation Patterns, PRL (2010)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.104.168104), Supplemental Material: compressed_cell_motion.avi

Converted the images to grayscale and an image sequence of tiffs

Attempting to replicate the $S(q,\omega)$ found in [T.E. Angelini *et al*, Glass-like dynamics of collective cell migration, PNAS (2011)](http://www.pnas.org/content/108/12/4714.abstract).


In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
# let matplotlib plot interactively in notebook
%matplotlib notebook

# Tweak styles
# mpl.rc('figure', figsize=(10,6))
mpl.rc('image',cmap='inferno')

import numpy as np
import scipy.integrate as integrate

import pims
import dynamicstructurefactor.sqw as sqw

savestuff = False

# Need some info of movies to get scales right
dt = 1 # one minute per frame
dx = 1.55 # 1.55 um/pixel

frames = pims.ImageSequence('compressed_cell_motion_frames/*.tif',as_grey=True)
frames

SyntaxError: invalid syntax (sqw.py, line 180)

In [ ]:
frameArray = sqw.image2array(frames)
framePSpec, [ww, qx, qy] = sqw.powerSpectrum(frameArray, spacings=[dx, dx, dt])

# Set up grid in fourier space
# [nqx, nqy, nw] = framePSpec.shape
# qx = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(nqx, dx))
# qy = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(nqy, dx))
# ww = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(nw,  dt))

In [ ]:
ww.size

Take a look at the spatial power spectrum

In [ ]:
fig1, ax1 = plt.subplots(1,1)
randw = int(np.floor(np.random.rand()*ww.size))
cax1 = ax1.pcolor(qx, qy, framePSpec[randw,...])
ax1.set_xlabel('$q_x \; (rad/\mu m)$')
ax1.set_ylabel('$q_y \; (rad/\mu m)$')
ax1.set_title('$|\mathcal{F}|^2(q_x, q_y)$, at $\omega$ = ' + str(randw) + ' rad/min')

cbar = fig1.colorbar(cax1)

if savestuff:
    fig1.savefig('2DPowerSpectrum.eps', format = 'eps')
    fig1.savefig('2DPowerSpectrum.tif', format = 'tif')

Take a look at some slice of the temporal fourier transform

In [ ]:
fig2, ax2 = plt.subplots(1,1)
randqx = int(np.floor(np.random.rand()*nqx))
randqy = int(np.floor(np.random.rand()*nqy))

ax2.plot(ww,framePSpec[randqx,randqy,:])
ax2.set_xlabel('$\omega \; (rad/min)$')
ax2.set_ylabel('$|\mathcal{F}|^2$')
ax2.set_title('$|\mathcal{F}|^2 (\omega)$ at $(q_x, q_y) = ($' + str(randqx) + ', ' + str(randqy) + ')')

if savestuff:
    fig2.savefig('randomOmegaSeries.eps', format = 'eps')
    fig2.savefig('randomOmegaSeries.tif', format = 'tif')

Perform azimuthal averaging and plot the resulting 2D spectrum

In [ ]:
aziAvg = sqw.azimuthalAverage3D(framePSpec)

In [ ]:
aziAvgOneSided = aziAvg[int(np.floor(nw/2)):-1,:]
qxOneSided = np.fft.ifftshift(qx)[:int(np.floor(qx.size/2))]
wwOneSided = np.fft.ifftshift(ww)[:int(np.floor(ww.size/2))]

fig3, ax3 = plt.subplots(1,1)
cax3 = ax3.pcolor(qxOneSided, wwOneSided, aziAvgOneSided[:wwOneSided.size, :qxOneSided.size])
ax3.set_xlabel('|q| (rad/$\mu$m)')
ax3.set_ylabel('$\omega$ (rad/min)')
ax3.set_title('$S(q,\omega)$, Angelini MDCK cells')

cbar = fig3.colorbar(cax3)

if savestuff:
    fig3.savefig('radialAvgSQW.eps', format = 'eps')
    fig3.savefig('radialAvgSQW.tif', format = 'tif')

Erm... not quite sure what to make of that.. Looks nothing like the S(q,$\omega$) in Angelini's PNAS paper. Looking for zeros at low *q* and high $\omega$..

I'm willing to bet that what needs to be done is subtracting the average intensity over all time and space to get rid of the huge peak at zero and actually see the real differences along the q and $\omega$

In [ ]:
frameArray_meanSub = np.ndarray.astype(frameArray, float) - frameArray.mean()
framePSpec = sqw.powerSpectrum(frameArray_meanSub, norm = False)

In [ ]:
fig, ax = plt.subplots(1,1)

cax1 = ax.pcolor(qx,qy,framePSpec[:,:,randw])
ax.set_xlabel('$q_x \; (rad/\mu m)$')
ax.set_ylabel('$q_y \; (rad/\mu m)$')
ax.set_title('Mean Subtracted $|\mathcal{F}|^2(q_x, q_y)$, at $\omega$ = ' + str(randw) + ' rad/min')

cbar = fig.colorbar(cax1)

if savestuff:
    fig1.savefig('2DPowerSpectrum_meanSub.eps', format = 'eps')
    fig1.savefig('2DPowerSpectrum_meanSub.tif', format = 'tif')

In [ ]:
# On second thought, I don't know what information this whole thing really gives us...

# fig4, ax4 =  plt.subplots(3,3)

# ax4[0,0].plot(framePSpec[1,1,:])
# t = ax4[0,0].yaxis.get_offset_text()
# t.set_x(-0.5)
# ax4[0,0].set_title('$(q_x,q_y)=$' + str(qxArr[1]) + ',' + str(qyArr[1]))

# ax4[0,1].plot(framePSpec[200,1,:])
# ax4[0,1].set_title(str(qxArr[200]) + ',' + str(qyArr[1]))

# ax4[0,2].plot(framePSpec[400,1,:])
# ax4[0,2].set_title(str(qxArr[400]) + ',' + str(qyArr[1]))

# ax4[1,0].plot(framePSpec[1,200,:])
# ax4[1,0].set_title(str(qxArr[1])   + ',' + str(qyArr[200]))

# ax4[1,1].plot(framePSpec[200,200,:])
# ax4[1,1].set_title(str(qxArr[200]) + ',' + str(qyArr[200]))

# ax4[1,2].plot(framePSpec[400,200,:])
# ax4[1,2].set_title(str(qxArr[400]) + ',' + str(qyArr[200]))

# ax4[2,0].plot(framePSpec[1,400,:])
# ax4[2,0].set_title(str(qxArr[1])   + ',' + str(qyArr[400]))

# ax4[2,1].plot(framePSpec[200,400,:])
# ax4[2,1].set_title(str(qxArr[200]) + ',' + str(qyArr[400]))

# ax4[2,2].plot(framePSpec[400,400,:])
# ax4[2,2].set_title(str(qxArr[400]) + ',' + str(qyArr[400]))

# fig4.subplots_adjust(hspace = 0.5, wspace = 0.5)
# fig4.suptitle('$\omega$ slices')

In [ ]:
aziAvg = sqw.azimuthalAverage3D(framePSpec, tdim=2)

In [ ]:
aziAvgOneSided = aziAvg[int(np.floor(nw/2)):-1,:]
    
fig5, ax5 = plt.subplots(1,1)
cax5 = ax5.pcolor(qxOneSided, wwOneSided, aziAvgOneSided[:wwOneSided.size, :qxOneSided.size])
ax5.set_xlabel('|q| (rad/$\mu$m)')
ax5.set_ylabel('$\omega$ (rad/min)')
ax5.set_title('$S(q,\omega)$, Angelini MDCK cells')

cbar = fig5.colorbar(cax5)

if savestuff:
    fig5.savefig('radialAvgSQW_meanSub.eps', format = 'eps')
    fig5.savefig('radialAvgSQW_meanSub.tif', format = 'tif')

So.. no difference

Let's try to divide by S(q) to get some better scales here.

$$ S(q) = \frac{1}{2 \pi} \int\limits_{-\infty}^{\infty} \; d \omega \;  S(q,\omega) $$

In [ ]:
sofcue = integrate.trapz(aziAvg, ww, axis=0)

sofcueFig, sofcueAx = plt.subplots()
sofcueAx.plot(sofcue)
sofcueAx.set_xlabel('$q \; (rad/\mu m)$')
sofcueAx.set_ylabel('$S(q)$')
sofcueAx.set_title('Static Structure Factor')

if savestuff:
    sofcueFig.savefig('sofq.eps',format = 'eps')
    sofcueFig.savefig('sofq.tif',format = 'tif')

In [ ]:
normed = aziAvg/sofcue[:,None].T

Quick excursion into using plotly's heatmap to maybe add some more interactivity into this...

In [ ]:
oneSidedNormed = normed[int(np.floor(170/2)):-1,:]

trace = go.Heatmap(z = oneSidedNormed[1:wwOneSided.size, 1:qxOneSided.size],
                  x = qxOneSided[1:],
                  y = wwOneSided[1:])
data = [trace]
layout = go.Layout(
    xaxis = dict(
        title = 'q (rad/um)',
        titlefont = dict(
            family = 'Helvetica, sans-serif',
            color = 'black')),
    yaxis = dict(
        title = 'w (rad/min)',
        titlefont = dict(
            family = 'Helvetica, sans-serif')))


heatMapFig = go.Figure(data=data, layout=layout)
iplot(heatMapFig, filename = 'S(q,w)/S(q)')

In [ ]:
heatFig, heatAx = plt.subplots(1,1)
caxHeat = heatAx.pcolor(np.log(oneSidedNormed))

heatAx.set_xlabel('|q|')
heatAx.set_ylabel('$\omega$')
heatAx.set_title('$\log(S(q,\omega)/S(q))$, Angelini MDCK cells')
heatAx.set_xlim(0,150)

cbarNormed = heatFig.colorbar(caxHeat)
cbarNormed.set_clim(vmax = 0.05)

if savestuff:
    heatFig.savefig('logNormedSQW.eps',format='eps')
    heatFig.savefig('logNormedSQW.tif',format='tif')

In [ ]:
heatFig, heatAx = plt.subplots(1,1)
caxHeat = heatAx.pcolor(qxOneSided[1:], wwOneSided[1:],
                        2*(oneSidedNormed[1:wwOneSided.size, 1:qxOneSided.size]), vmax=1.5)

heatAx.set_xlabel('|q|')
heatAx.set_ylabel('$\omega$')
heatAx.set_title('$S(q,\omega)/S(q)$, Angelini MDCK cells')
#heatAx.set_xlim(0,150)

cbarNormed = heatFig.colorbar(caxHeat,extend='max')
#cbarNormed.set_clim(vmax = 1.0)
#cbarNormed.set_ticks([0.0,0.02,0.04,0.06,0.08,0.1])

if savestuff:
    heatFig.savefig('NormedSQW_highContrast.eps',format='eps')
    heatFig.savefig('NormedSQW_highContrast.tif',format='tif')

In [ ]:
np.savetxt('normedSQW.txt', normed, header = 'S(q,w)/S(q) for Angelini data')
np.savetxt('qArray.txt', qxOneSided, header = 'Wave vectors for Angelini data')
np.savetxt('wArray.txt', ww, header = 'Frequencies for Angelini data')
np.savetxt('sqw.txt', aziAvg, header = 'Azimuthally averaged S(q,w) for Angelini data')

Okay great! This gives something that look WAY better. And doesn't need to be logged in order to see the structure inherent in there.

I do seem to be off in scale for this, but that could be a question about normalizations that I need to be more careful about, multiplying by 2 in getting the one sided power spectral density, etc. I think these are questions that are better left for a later date, and it's time to try and see if we get anything interesting for the real stuff.